<a id='Top'></a>

# MultiSurv results<a class='tocSkip'></a>

Evaluation metric results for MultiSurv.

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext watermark

import sys
import os

import ipywidgets as widgets
import numpy as np
import pandas as pd
import torch

# Make modules in "src" dir visible
project_dir = os.path.split(os.getcwd())[0]
if project_dir not in sys.path:
    sys.path.append(os.path.join(project_dir, 'src'))

import dataset
from model import Model
import utils

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DataLoader" data-toc-modified-id="DataLoader-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><code>DataLoader</code></a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#Load-weights" data-toc-modified-id="Load-weights-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load weights</a></span></li></ul></li><li><span><a href="#Evaluate" data-toc-modified-id="Evaluate-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Evaluate</a></span><ul class="toc-item"><li><span><a href="#Write-to-results-table" data-toc-modified-id="Write-to-results-table-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Write to results table</a></span></li><li><span><a href="#Check-results-on-all-datasets" data-toc-modified-id="Check-results-on-all-datasets-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Check results on all datasets</a></span></li></ul></li></ul></div>

In [2]:
DATA = utils.INPUT_DATA_DIR
MODELS = utils.TRAINED_MODEL_DIR

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# `DataLoader`

In [3]:
modalities = widgets.SelectMultiple(
    options=['clinical', 'mRNA', 'DNAm', 'miRNA', 'CNV', 'wsi'],
    index=[0, 1],
    rows=6,
    description='Input data',
    disabled=False
)
display(modalities)

SelectMultiple(description='Input data', index=(0, 1), options=('clinical', 'mRNA', 'DNAm', 'miRNA', 'CNV', 'w…

In [5]:
dataloaders = utils.get_dataloaders(
    data_location=DATA,
    labels_file='../data/labels.tsv',
    modalities=modalities.value,
    wsi_patch_size=299,
    n_wsi_patches=5,
#     exclude_patients=exclude_cancers,
)

Data modalities:
   clinical
   mRNA

Dataset sizes (# patients):
   train: 8880
   val: 1109
   test: 1092

Batch size: 128


# Model

In [6]:
multisurv = Model(dataloaders=dataloaders,
#                   output_intervals=prediction_intervals,
                  device=device)

Instantiating MultiSurv model...


## Load weights

In [7]:
!ls -1 /mnt/dataA/multisurv_models/

clinical_CNV_lr0.005_epoch46_acc0.77.pth
clinical_DNAm_lr0.005_20_cancers_epoch39_concord0.79.pth
clinical_DNAm_lr0.005_epoch48_acc0.80.pth
clinical_lr0.005_epoch49_acc0.78.pth
clinical_miRNA_lr0.005_epoch45_acc0.78.pth
clinical_mRNA_DNAm_lr0.0005pre_epoch39_acc0.80.pth
clinical_mRNA_DNAm_lr0.005a_32_epoch54_acc0.80.pth
clinical_mRNA_DNAm_lr0.005a_64_epoch40_acc0.81.pth
clinical_mRNA_DNAm_lr0.005a_epoch32_acc0.80.pth
clinical_mRNA_DNAm_lr0.005b_32_epoch57_acc0.80.pth
clinical_mRNA_DNAm_lr0.005b_epoch31_acc0.80.pth
clinical_mRNA_DNAm_lr0.005c_epoch40_acc0.80.pth
clinical_mRNA_DNAm_lr0.005d_epoch13_acc0.80.pth
clinical_mRNA_DNAm_lr0.005d_epoch43_acc0.79.pth
clinical_mRNA_DNAm_lr0.005e_epoch30_acc0.80.pth
clinical_mRNA_lr0.001_TEST_LOSS_REDUCTION_epoch51_concord0.80.pth
clinical_mRNA_lr0.001_TEST_LOSS_REDUCTION_epoch75_concord0.80.pth
clinical_mRNA_lr0.005_10_1year_intervals_epoch37_acc0.80.pth
clinical_mRNA_lr0.005_10_half-year_intervals_epoch35_acc0.79.pth
clinical_mRNA_lr0.005_20_1year

In [8]:
!ls -1 /mnt/dataA/multisurv_models/wsi*

/mnt/dataA/multisurv_models/wsi5patches299px_lr0.001_epoch44_concord0.55.pth


In [9]:
# Best model
multisurv.load_weights(os.path.join(MODELS, 'clinical_mRNA_lr0.005_epoch43_acc0.81.pth'))

Load model weights:
/mnt/dataA/multisurv_models/clinical_mRNA_lr0.005_epoch43_acc0.81.pth


In [10]:
for modality in modalities.value:
    print(modality)

clinical
mRNA


# Evaluate

In [68]:
%%time

# Using MultiSurv's default output intervals
performance = utils.Evaluation(model=multisurv, dataset=dataloaders['test'].dataset,
                               device=device)
performance.run_bootstrap()
print()

Collect patient predictions: 1092/1092

Bootstrap
---------
1000/1000

CPU times: user 3min 30s, sys: 6.51 s, total: 3min 36s
Wall time: 3min 27s


In [69]:
data_modalities = ' + '.join(modalities.value) if len(modalities.value) > 1 else modalities.value[0]
print(f'>> {data_modalities} <<')
print()
performance.show_results()

>> clinical + mRNA <<

          Value (95% CI)
-----------------------------
C-index   0.818 (0.801-0.834)
Ctd       0.822 (0.806-0.837)
IBS       0.138 (0.126-0.15)
INBLL     0.425 (0.393-0.459)


In [70]:
data_modalities = ' + '.join(modalities.value) if len(modalities.value) > 1 else modalities.value[0]
print(f'>> {data_modalities} <<')
print()
performance.show_results(method='empirical')

>> clinical + mRNA <<

          Value (95% CI)
-----------------------------
C-index   0.818 (0.801-0.834)
Ctd       0.822 (0.806-0.837)
IBS       0.138 (0.126-0.15)
INBLL     0.425 (0.393-0.459)


## Write to results table

In [29]:
results = utils.ResultTable()

In [34]:
data_modalities = ' + '.join(modalities.value) if len(modalities.value) > 1 else modalities.value[0]

results.write_result_dict(result_dict=performance.format_results(),
                          algorithm='MultiSurv',
                          data_modality=data_modalities)
results.table

clinical                 mRNA  \
Algorithm     Metric                                              
CPH           C-index  0.796 (0.779-0.813)  0.733 (0.712-0.756)   
              Ctd      0.796 (0.779-0.813)  0.733 (0.712-0.755)   
              IBS      0.143 (0.135-0.154)   0.177 (0.165-0.19)   
              INBLL    0.438 (0.414-0.465)  0.528 (0.497-0.558)   
RSF           C-index  0.764 (0.744-0.782)  0.718 (0.695-0.741)   
              Ctd       0.77 (0.751-0.789)  0.719 (0.695-0.741)   
              IBS      0.184 (0.179-0.191)    0.191 (0.181-0.2)   
              INBLL     0.546 (0.533-0.56)  0.563 (0.537-0.581)   
DeepSurv      C-index  0.792 (0.773-0.811)  0.746 (0.722-0.768)   
              Ctd       0.792 (0.773-0.81)  0.746 (0.722-0.768)   
              IBS      0.143 (0.134-0.154)   0.18 (0.159-0.198)   
              INBLL    0.436 (0.412-0.464)   0.606 (0.513-0.68)   
CoxTime       C-index  0.789 (0.771-0.806)  0.731 (0.707-0.753)   
              Ctd        0.8 (0.783-0.817)  0.732 (0.708-0.755)   
              IBS      0.148 (0.139-0.159)  0.173 (0.159-0.185)   
              INBLL    0.448 (0.427-0.475)  0.524 (0.488-0.556)   
DeepHit       C-index   0.798 (0.78-0.815)   0.744 (0.72-0.767)   
              Ctd      0.809 (0.792-0.826)  0.752 (0.728-0.774)   
              IBS      0.173 (0.165-0.184)   0.191 (0.18-0.198)   
              INBLL    0.517 (0.499-0.542)  0.559 (0.532-0.575)   
MTLR          C-index  0.801 (0.782-0.821)   0.742 (0.72-0.764)   
              Ctd      0.806 (0.787-0.825)  0.745 (0.723-0.766)   
              IBS      0.147 (0.134-0.161)  0.178 (0.156-0.196)   
              INBLL     0.441 (0.41-0.478)  0.543 (0.472-0.598)   
Nnet-survival C-index  0.805 (0.787-0.822)   0.69 (0.662-0.718)   
              Ctd       0.81 (0.794-0.828)  0.701 (0.673-0.728)   
              IBS      0.132 (0.122-0.147)  0.195 (0.169-0.212)   
              INBLL     0.404 (0.38-0.443)  0.724 (0.597-0.832)   
MultiSurv     C-index   0.806 (0.79-0.823)  0.755 (0.733-0.777)   
              Ctd      0.809 (0.793-0.825)   0.758 (0.735-0.78)   
              IBS      0.143 (0.134-0.155)  0.178 (0.157-0.194)   
              INBLL    0.439 (0.416-0.469)  0.533 (0.475-0.577)   

                                      DNAm                miRNA  \
Algorithm     Metric                                              
CPH           C-index   0.739 (0.719-0.76)    0.676 (0.651-0.7)   
              Ctd       0.739 (0.719-0.76)    0.676 (0.651-0.7)   
              IBS      0.179 (0.165-0.192)  0.186 (0.171-0.202)   
              INBLL    0.532 (0.499-0.563)  0.547 (0.511-0.585)   
RSF           C-index  0.728 (0.707-0.751)  0.663 (0.638-0.688)   
              Ctd      0.729 (0.709-0.752)  0.664 (0.639-0.689)   
              IBS      0.186 (0.176-0.192)  0.193 (0.183-0.201)   
              INBLL     0.55 (0.527-0.564)  0.567 (0.543-0.583)   
DeepSurv      C-index     0.76 (0.74-0.78)  0.685 (0.661-0.711)   
              Ctd       0.759 (0.739-0.78)  0.685 (0.661-0.711)   
              IBS      0.177 (0.156-0.203)   0.194 (0.17-0.219)   
              INBLL    0.931 (0.684-1.132)   0.61 (0.527-0.689)   
CoxTime       C-index  0.747 (0.726-0.767)  0.692 (0.667-0.718)   
              Ctd      0.747 (0.726-0.767)    0.694 (0.67-0.72)   
              IBS      0.172 (0.156-0.191)  0.174 (0.158-0.189)   
              INBLL    0.519 (0.475-0.571)  0.521 (0.479-0.563)   
DeepHit       C-index  0.733 (0.712-0.754)  0.701 (0.675-0.724)   
              Ctd      0.737 (0.715-0.758)    0.7 (0.674-0.725)   
              IBS      0.194 (0.179-0.208)  0.186 (0.177-0.197)   
              INBLL    0.571 (0.529-0.605)  0.548 (0.524-0.575)   
MTLR          C-index   0.747 (0.726-0.77)  0.691 (0.664-0.717)   
              Ctd       0.751 (0.73-0.775)  0.693 (0.666-0.718)   
              IBS       0.176 (0.162-0.19)  0.192 (0.169-0.214)   
              INBLL    0.542 (0.504-0.584)  0.589 (0.515-0.657)   
Nnet-survival

## Check results on all datasets

In [12]:
%%time

print('~' * 23)
print('     RESULT CHECK')
print('~' * 23)
check_results = {'train': None, 'val': None, 'test': None}

for group in check_results.keys():
    print(f'~ {group} ~')
    performance = utils.Evaluation(model=multisurv, dataset=dataloaders[group].dataset, device=device)
    performance.compute_metrics()
    performance.show_results()
    print()

~~~~~~~~~~~~~~~~~~~~~~~
     RESULT CHECK
~~~~~~~~~~~~~~~~~~~~~~~
~ train ~
Collect patient predictions: 8880/8880

C-index   0.843
Ctd       0.85
IBS       0.112
INBLL     0.349

~ val ~
Collect patient predictions: 1109/1109

C-index   0.805
Ctd       0.808
IBS       0.137
INBLL     0.423

~ test ~
Collect patient predictions: 1092/1092

C-index   0.818
Ctd       0.822
IBS       0.138
INBLL     0.425

CPU times: user 5min 36s, sys: 1min 18s, total: 6min 55s
Wall time: 13min 54s


# Watermark<a class='tocSkip'></a>

In [13]:
%watermark --iversions
%watermark -v
print()
%watermark -u -n

pandas     1.0.1
torch      1.4.0
numpy      1.18.1
ipywidgets 7.5.1

CPython 3.6.7
IPython 7.11.1

last updated: Tue Jul 28 2020


[Top of the page](#Top)